In [36]:
from docx import Document
import json
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from uuid import uuid4
from langchain_core.documents import Document as LangchainDocument

# Путь к файлу
infile = 'sample3.docx'

# Открываем документ
doc = Document(infile)

# Сбор текста, таблиц и списков
all_structured_data = []  # Для хранения всех списков и таблиц

# Проверка, является ли параграф частью списка
def is_list_paragraph(paragraph):
    style_name = paragraph.style.name.lower()
    return "list" in style_name or "bullet" in style_name or "number" in style_name

current_list = []  # Временное хранилище для текущего списка

for paragraph in doc.paragraphs:
    text = paragraph.text.strip()

    if is_list_paragraph(paragraph):
        if text:
            current_list.append(text)  # Добавляем элемент в текущий список
    else:
        if current_list:  # Если список закончился, сохраняем его
            all_structured_data.append({"type": "list", "content": current_list})
            current_list = []

        if text:  # Добавляем обычный текст
            continue

# Сохраняем последний список, если он остался
if current_list:
    all_structured_data.append({"type": "list", "content": current_list})

# Обработка таблиц
for table in doc.tables:
    table_data = []
    for row in table.rows:
        row_data = [cell.text.strip() for cell in row.cells]
        table_data.append(row_data)

    all_structured_data.append({"type": "table", "content": table_data})

# Конвертируем данные в формат документов для эмбеддинга
data_documents = [
    LangchainDocument(
        page_content=json.dumps(item["content"], ensure_ascii=False),
        metadata={"type": item["type"]}
    ) for item in all_structured_data
]

# Создаем базу данных FAISS с эмбеддингами
db = FAISS.from_documents(data_documents, OllamaEmbeddings(model="nomic-embed-text", show_progress=True))
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

# Настройка локальной модели
local_model = "llama3"
llm = ChatOllama(model=local_model, num_predict=400,
                 stop=["<|start_header_id|>", "<|end_header_id|>", "<|eot_id|>"])

# Настройка цепочки RAG
prompt_template = """
<|start_header_id|>user<|end_header_id|>
Responds user questions taking into account the given context, give a precise and short answer without referring to this part of the prompt.
Give as little additional information as possible.
Question: {question}
Context: {context}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Тестирование RAG-цепочки
question = "What share percent have VoiceOver screen reader?"
print("Question:", question)
res = rag_chain.invoke(question)
print("Answer:", res)

# Сохранение всех структурированных данных в файл
with open("document_structure.json", "w", encoding="utf-8") as f:
    json.dump(all_structured_data, f, ensure_ascii=False, indent=4)

print("Структура документа сохранена в 'document_structure.json'")


OllamaEmbeddings: 100%|██████████| 3/3 [00:06<00:00,  2.09s/it]


Question: What share percent have VoiceOver screen reader?


OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.07s/it]


Answer: 9%
Структура документа сохранена в 'document_structure.json'


In [40]:
evaluator = EvaluationMetrics()

reference = '9%'
results = evaluator.evaluate([reference],[res])

In [41]:
print("ROUGE scores")
pd.DataFrame(results["rouge"])

ROUGE scores


,rouge-1,rouge-2,rouge-l
r,1.0,0.0,1.0
p,1.0,0.0,1.0
f,1.0,0.0,1.0


In [42]:
print("BERTScore")
pd.DataFrame(results["bertscore"], index=['score']).T

BERTScore


,score
precision,1.0
recall,1.0
f1,1.0
